In [1]:
import pandas as pd
from slugify import slugify
import os
import glob
import re
import numpy as np
from rapidfuzz import fuzz
from geopy.distance import geodesic

In [2]:
raw_dir = "../data/raw"
clean_dir = "../data/clean"
os.makedirs(clean_dir, exist_ok=True)

## Data Cleaning Functions

In [3]:
def strip_string_cells(df: pd.DataFrame) -> pd.DataFrame:
    """Strip whitespace from all string columns"""
    df = df.copy()
    for col in df.select_dtypes(include=["object", "string"]).columns:
        df[col] = df[col].astype(str).str.strip()
    return df

In [4]:
def clean_column_names(df: pd.DataFrame) -> pd.DataFrame:
    """Standardize column names: lowercase, underscore-separated"""
    df = df.copy()
    df.columns = [
        re.sub(r"\s+", "_", col.strip().lower())
        for col in df.columns
    ]
    return df

In [5]:
def canonicalize_address(addr: str) -> str:
    """Normalize addresses for matching"""
    if pd.isna(addr):
        return ""
    s = str(addr).strip().lower()

    # Remove unit/suite markers
    s = s.replace("#", " ")      
    s = s.replace(" unit ", " ")
    s = s.replace(" ste ", " ")
    s = s.replace(" suite ", " ")

    # Standardize street suffixes
    suffix_map = {
        " st": " street",
        " st,": " street",
        " st.": " street",
        " rd": " road",
        " rd.": " road",
        " ave": " avenue",
        " ave.": " avenue",
        " ave,": " avenue",
        " blvd": " boulevard",
        " dr": " drive",
        " dr.": " drive",
        " plz": " plaza",
        " pl": " place",
        " hwy": " highway",
    }

    for old, new in suffix_map.items():
        if s.endswith(old):
            s = s[: -len(old)] + new
        s = s.replace(old + " ", new + " ")

    # Normalize whitespace and slugify
    s = " ".join(s.split())
    s = slugify(s, lowercase=True, separator=' ')

    return s

In [6]:
def canonicalize_name(name: str) -> str:
    """Normalize restaurant names for matching"""
    if pd.isna(name):
        return ""
    s = str(name).strip().lower()
    
    # Remove common suffixes/prefixes
    patterns_to_remove = [
        r"\s*-\s*walk\s*thru\s*$",
        r"\s*restaurant\s*$",
        r"\s*cafe\s*$",
        r"\s*inc\s*$",
        r"\s*llc\s*$",
        r"\s*\(.*\)\s*",  # Remove parenthetical content
    ]
    
    for pattern in patterns_to_remove:
        s = re.sub(pattern, "", s, flags=re.IGNORECASE)
    
    # Normalize and slugify
    s = " ".join(s.split())
    s = slugify(s, lowercase=True, separator=' ')
    
    return s

In [7]:
def clean_inspection_df(df: pd.DataFrame) -> pd.DataFrame:
    """Clean and standardize inspection dataframe"""
    df = clean_column_names(df)
    df = strip_string_cells(df)

    # Parse dates
    date_cols = ["inspection_date", "data_as_of", "data_loaded_at"]
    for col in date_cols:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], errors="coerce")

    # Convert coordinates to numeric
    for col in ["latitude", "longitude", "lat", "lng"]:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors="coerce")

    # Create canonical address
    source_col = None
    if "street_address_clean" in df.columns:
        source_col = "street_address_clean"
    elif "address" in df.columns:
        source_col = "address"

    if source_col is not None:
        df["canonical_address"] = df[source_col].apply(canonicalize_address)
    
    # Create canonical name
    if "dba" in df.columns:
        df["canonical_name"] = df["dba"].apply(canonicalize_name)
    elif "name" in df.columns:
        df["canonical_name"] = df["name"].apply(canonicalize_name)

    return df

## Load and Clean Individual Datasets

In [8]:
# Load SF Health Inspection Data
sf_path = os.path.join(raw_dir, "Health_Inspection_Scores_(2024-Present)_20251030.csv")
df_sf = pd.read_csv(sf_path, encoding="latin1")
df_sf = clean_inspection_df(df_sf)

print("SF Inspection Data Shape:", df_sf.shape)
print("\nColumns:", df_sf.columns.tolist())
display(df_sf.head(3))

/var/folders/ty/hk0ht40x61710dlz73sh5bz80000gn/T/ipykernel_14914/1479803450.py:10: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], errors="coerce")


SF Inspection Data Shape: (19056, 28)

Columns: ['inspection_date', 'inspector', 'district', 'subdistrict', 'subsector', 'permit_number', 'dba', 'permit_type', 'street_address', 'street_address_clean', 'inspection_type', 'inspection_frequency_type', 'total_time', 'facility_rating_status', 'census', 'suspension_notes', 'inspection_notes', 'violation_count', 'violation_codes', 'latitude', 'longitude', 'point', 'analysis_neighborhood', 'supervisor_district', 'data_as_of', 'data_loaded_at', 'canonical_address', 'canonical_name']


,inspection_date,inspector,district,subdistrict,subsector,permit_number,dba,permit_type,street_address,street_address_clean,...,violation_codes,latitude,longitude,point,analysis_neighborhood,supervisor_district,data_as_of,data_loaded_at,canonical_address,canonical_name
0,2025-04-23,Michael Mooney,1,103,607,06734928,Surfside - Walk Thru,H36 - STADIUM CONCESSIONS (PERM),24 WILLIE MAYS PLZ # PROMEN,3RD ST & KING ST,...,nan,37.778130,-122.391855,POINT (-122.391855 37.77813),Mission Bay,6.0,2025-07-01 10:09:15,2025-10-30 02:37:07,3rd street king street,surfside
1,2025-04-23,Michael Mooney,2,201,106,06735187,HARBOR EMPEROR,H33 - COMMISSARIES,41 EMBARCADERO,41 EMBARCADERO,...,"113953(c), 114163(a)(3), 114189, 114192.1, 114...",37.787126,-122.387925,POINT (-122.387924588 37.787126305),Financial District/South Beach,6.0,2025-07-01 10:09:15,2025-10-30 02:37:07,41 embarcadero,harbor emperor
2,2025-04-23,Patrick Wood,1,101,176A,06743776,RA @ BLOOMBERG FLOOR 22,H85 - EMPLOYEE CAFETERIA LIMITED FOOD PREP,140 NEW MONTGOMERY ST FL 22,140 NEW MONTGOMERY ST,...,nan,37.786617,-122.400018,POINT (-122.400018 37.786617),Financial District/South Beach,6.0,2025-07-01 10:09:15,2025-10-30 02:37:07,140 new montgomery street,ra bloomberg floor 22


In [9]:
# Load Google Restaurant Data
google_path = os.path.join(raw_dir, "sf_restaurants_google.csv")
df_google = pd.read_csv(google_path, encoding="latin1")
df_google = clean_inspection_df(df_google)

print("Google Restaurant Data Shape:", df_google.shape)
print("\nColumns:", df_google.columns.tolist())
display(df_google.head(3))

Google Restaurant Data Shape: (1970, 13)

Columns: ['name', 'address', 'rating', 'user_ratings_total', 'types', 'open_now', 'lat', 'lng', 'place_id', 'business_status', 'reviews', 'canonical_address', 'canonical_name']


,name,address,rating,user_ratings_total,types,open_now,lat,lng,place_id,business_status,reviews,canonical_address,canonical_name
0,Piccolo Forno,"725 Columbus Ave, San Francisco, CA 94133, Uni...",4.7,2013.0,"restaurant, food, point_of_interest, establish...",False,37.801380,-122.411986,ChIJLeYyO_CAhYAR6j9XsEIm3GI,OPERATIONAL,"[{'author': 'William Irwin', 'rating': 5, 'tex...",725 columbus avenue san francisco ca 94133 uni...,piccolo forno
1,Bottega,"1132 Valencia St, San Francisco, CA 94110, Uni...",4.4,990.0,"bar, store, restaurant, food, point_of_interes...",False,37.754715,-122.421194,ChIJxZMBuHV_j4ARK1fO7KSqyOE,OPERATIONAL,"[{'author': 'Moorissa Tjokro', 'rating': 4, 't...",1132 valencia street san francisco ca 94110 un...,bottega
2,Cotogna,"490 Pacific Ave, San Francisco, CA 94133, Unit...",4.5,2080.0,"restaurant, food, point_of_interest, establish...",False,37.797399,-122.403576,ChIJA0YvGPWAhYAReXmaDTTdWzU,OPERATIONAL,"[{'author': 'Pavlo Debelynskyi', 'rating': 5, ...",490 pacific avenue san francisco ca 94133 unit...,cotogna


In [21]:
# Save cleaned individual datasets
df_sf.to_csv(os.path.join(clean_dir, "sfData_cleaned.csv"), index=False)
df_google.to_csv(os.path.join(clean_dir, "googleData_cleaned.csv"), index=False)

print("Cleaned datasets saved successfully.")


Cleaned datasets saved successfully.


## Matching Functions

In [11]:
def calculate_geo_distance(lat1, lon1, lat2, lon2):
    """Calculate distance in meters between two coordinates"""
    if pd.isna(lat1) or pd.isna(lon1) or pd.isna(lat2) or pd.isna(lon2):
        return np.nan
    try:
        return geodesic((lat1, lon1), (lat2, lon2)).meters
    except:
        return np.nan

In [12]:
def find_best_match(sf_row, df_google, use_geo=True, geo_threshold_m=100, 
                    name_threshold=80, address_threshold=85):
    """
    Find best matching Google restaurant for an SF inspection record.
    
    Matching strategy:
    1. Geographic filter: Only consider restaurants within geo_threshold_m meters
    2. Score combinations of name and address similarity
    3. Return best match above thresholds
    """
    candidates = df_google.copy()
    
    # Filter by geographic proximity if coordinates available
    if use_geo and not pd.isna(sf_row['latitude']) and not pd.isna(sf_row['longitude']):
        candidates['geo_distance'] = candidates.apply(
            lambda x: calculate_geo_distance(
                sf_row['latitude'], sf_row['longitude'],
                x['lat'], x['lng']
            ), axis=1
        )
        candidates = candidates[candidates['geo_distance'] <= geo_threshold_m]
        
        if len(candidates) == 0:
            return None, None, None
    
    # Calculate name similarity
    candidates['name_score'] = candidates['canonical_name'].apply(
        lambda x: fuzz.ratio(sf_row['canonical_name'], x) if pd.notna(x) else 0
    )
    
    # Calculate address similarity
    candidates['address_score'] = candidates['canonical_address'].apply(
        lambda x: fuzz.ratio(sf_row['canonical_address'], x) if pd.notna(x) else 0
    )
    
    # Combined score: weighted average
    candidates['combined_score'] = (
        0.6 * candidates['name_score'] + 
        0.4 * candidates['address_score']
    )
    
    # Find best match
    best_idx = candidates['combined_score'].idxmax()
    best_match = candidates.loc[best_idx]
    
    # Apply thresholds
    if (best_match['name_score'] >= name_threshold or 
        best_match['address_score'] >= address_threshold):
        return best_match['place_id'], best_match['combined_score'], best_idx
    
    return None, None, None

## Merge Datasets

In [13]:
# Prepare SF data for matching
df_sf_subset = df_sf[[
    'permit_number', 'dba', 'canonical_name', 'street_address', 
    'canonical_address', 'latitude', 'longitude', 'inspection_date',
    'facility_rating_status', 'violation_count'
]].copy()

# Initialize matching columns
df_sf_subset['matched_place_id'] = None
df_sf_subset['match_score'] = None

print(f"Starting matching process for {len(df_sf_subset)} SF records...")

Starting matching process for 19056 SF records...


In [14]:
# Perform matching with progress tracking
from tqdm import tqdm

matched_count = 0

for idx, row in tqdm(df_sf_subset.iterrows(), total=len(df_sf_subset)):
    place_id, score, match_idx = find_best_match(
        row, df_google, 
        use_geo=True, 
        geo_threshold_m=150,
        name_threshold=75,
        address_threshold=80
    )
    
    if place_id is not None:
        df_sf_subset.at[idx, 'matched_place_id'] = place_id
        df_sf_subset.at[idx, 'match_score'] = score
        matched_count += 1

print(f"\nMatched {matched_count} out of {len(df_sf_subset)} records")
print(f"Match rate: {matched_count / len(df_sf_subset) * 100:.2f}%")

100%|██████████| 19056/19056 [29:00<00:00, 10.95it/s] 


Matched 3024 out of 19056 records
Match rate: 15.87%


In [22]:
# Merge with Google data
df_merged = df_sf_subset.merge(
    df_google[['place_id', 'name', 'address', 'rating', 'user_ratings_total', 
               'types', 'lat', 'lng', 'business_status']],
    left_on='matched_place_id',
    right_on='place_id',
    how='left',
    suffixes=('_sf', '_google')
)

print("Merged Dataset Shape:", df_merged.shape)
display(df_merged.head(100))

Merged Dataset Shape: (19056, 21)


,permit_number,dba,canonical_name,street_address,canonical_address,latitude,longitude,inspection_date,facility_rating_status,violation_count,...,match_score,place_id,name,address,rating,user_ratings_total,types,lat,lng,business_status
0,06734928,Surfside - Walk Thru,surfside,24 WILLIE MAYS PLZ # PROMEN,3rd street king street,37.778130,-122.391855,2025-04-23,Pass,NaN,...,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,06735187,HARBOR EMPEROR,harbor emperor,41 EMBARCADERO,41 embarcadero,37.787126,-122.387925,2025-04-23,Conditional Pass,4.0,...,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,06743776,RA @ BLOOMBERG FLOOR 22,ra bloomberg floor 22,140 NEW MONTGOMERY ST FL 22,140 new montgomery street,37.786617,-122.400018,2025-04-23,Pass,NaN,...,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,102419,MOKUKU,mokuku,332 CLEMENT ST,332 clement street,37.783269,-122.462932,2025-04-23,Pass,NaN,...,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,06734548,STEEP CREAMERY & TEA - SECTION 110,steep creamery tea section 110,24 WILLIE MAYS PLZ,3rd street king street,37.778130,-122.391855,2025-04-23,Pass,NaN,...,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,A99312,LOST AND FOUND SF,lost and found sf,1439 TARAVAL ST,1439 taraval street,37.742544,-122.481752,2025-04-09,Closure,3.0,...,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,18306,FERRY PLAZA FARMERS MARKET,ferry plaza farmers market,1 FERRY BUILDING 50,the embarcadero market street,37.794935,-122.394265,2025-04-08,Pass,NaN,...,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,88498,MCDONALD'S,mcdonald s,441 SUTTER ST,441 sutter street,37.789065,-122.407650,2025-04-08,Pass,2.0,...,79.15493,ChIJRagBt46AhYARJim-yIO5cyo,McDonald's,"441 Sutter St, San Francisco, CA 94108, United...",3.8,2527.0,"cafe, store, restaurant, food, point_of_intere...",37.789217,-122.407641,OPERATIONAL
98,51055,LUCKY 756,lucky 756,1750 FULTON ST,1750 fulton street,37.776168,-122.445792,2025-04-08,Pass,NaN,...,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
# Analyze match quality
print("Match Quality Analysis:")
print(f"Total records: {len(df_merged)}")
print(f"Matched records: {df_merged['matched_place_id'].notna().sum()}")
print(f"Unmatched records: {df_merged['matched_place_id'].isna().sum()}")
print(f"\nMatch Score Statistics:")
print(df_merged['match_score'].describe())

Match Quality Analysis:
Total records: 19056
Matched records: 3024
Unmatched records: 16032

Match Score Statistics:
count     3024.000000
unique     326.000000
top         78.550725
freq       453.000000
Name: match_score, dtype: float64


In [17]:
# Save merged dataset
output_path = os.path.join(clean_dir, "merged_sf_google_data.csv")
df_merged.to_csv(output_path, index=False)
print(f"Merged dataset saved to: {output_path}")

Merged dataset saved to: ../data/clean/merged_sf_google_data.csv


## Explore Merged Data

In [28]:
# Sample matched records

matched_sample = df_merged[df_merged['matched_place_id'].notna()].head(50)
display(matched_sample[['dba', 'name', 'rating', 'facility_rating_status', 
                        'violation_count', 'match_score']])


,dba,name,rating,facility_rating_status,violation_count,match_score
10,KHAO TIEW,Khao Tiew,4.6,Pass,5.0,80.512821
19,SUM DIM SUM,Sum Dim Sum,3.8,Pass,2.0,78.550725
22,TIN ON KEARNY,Tin on Kearny,4.9,Pass,7.0,79.15493
31,HAPPY KING PIZZA,Happy king pizza,4.6,Pass,5.0,77.910448
42,HOUSE OF PRIME RIB,House of Prime Rib,4.7,nan,NaN,80.779221
60,LA VACA BIRRIA,La Vaca Birria,4.4,Pass,6.0,78.550725
65,ZITOUNA,Zitouna,4.8,Pass,NaN,80.266667
68,MATADOR,Matador,4.3,Pass,5.0,79.15493
75,FRANCO'S LATIN TABLE,Franco's Latin Table,4.3,Pass,NaN,78.947368
77,TRISARA RESTAURANT,Trisara,4.7,Pass,NaN,79.15493
